In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Differentially-Private-Fine-tuning-of-Language-Models-main/Language-Generation-GPT-2')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#used these versions since they where used by the differential private finetuning code
!pip install opacus==0.15.0
#!pip install torch==1.11.0
!pip install datasets
!pip install loralib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, get_scheduler, DataCollatorForLanguageModeling
from opacus import PrivacyEngine
from datasets import load_dataset
import torch.nn.functional as F
#######################
import os
from os.path import join, abspath, dirname
from data_utils.dataset import load_file, LAMADataset
from data_utils.vocab import init_vocab
from p_tune.modeling import PTuneForLAMA
from transformers import AutoTokenizer
#############################
from loralib import MergedLinear
import loralib as lora
from opacus.grad_sample import utils as opacus_utils
from opacus.layers import DPLSTM
from tqdm import tqdm
from src.model import GPT2Config, GPT2LMModel
from src.data_utils import FT_Dataset

In [5]:
#functions used in this code
def get_task_name():
        if args.only_evaluate:
            return "_".join([args.model_name + ('_' + args.vocab_strategy), 'only_evaluate'])
        names = [args.model_name + ('_' + args.vocab_strategy),
                 "template_{}".format(args.template if not args.use_original_template else 'original'),
                 "fixed" if not args.use_lm_finetune else "fine-tuned",
                 "seed_{}".format(args.seed)]
        return "_".join(names)

def get_TREx_parameters():
        relation = load_file(join(args.data_dir, "single_relations/{}.jsonl".format(args.relation_id)))[0]
        data_path_pre = "fact-retrieval/original/{}/".format(args.relation_id)
        data_path_post = ".jsonl"
        return relation, data_path_pre, data_path_post
def get_save_path():
        return join(args.out_dir, 'prompt_model', args.model_name, 'search', get_task_name(),
                    args.relation_id)

def reverse_zero_pad(x, W, enable_lora, out_features):
    lora_ind = W.new_zeros((out_features, ), dtype=torch.bool).view(len(enable_lora), -1)
    lora_ind[enable_lora, :] = True
    lora_ind = lora_ind.view(-1)
    result = x.new_zeros((*x.shape[:-1], out_features // len(enable_lora) * sum(enable_lora)))
    result = result.view(-1, out_features // len(enable_lora) * sum(enable_lora))
    result = x.reshape(-1, out_features)[:, lora_ind]
    return result.view((*x.shape[:-1], out_features // len(enable_lora) * sum(enable_lora)))


def compute_transformers_MergedLinear_grad_sample(layer: MergedLinear, A: torch.Tensor, B: torch.Tensor, batch_dim: int = 0) -> None:
    delta1 = reverse_zero_pad(B, layer.weight, layer.enable_lora, layer.out_features) * layer.scaling
    after_A = F.linear(layer.lora_dropout(A), layer.lora_A)
    t_after_A = after_A.transpose(-2, -1)
    in_channel = t_after_A.shape[1]
    out_channel = delta1.shape[-1]
    lora_b_channel = layer.lora_B.shape[0]

    gs1 = torch.einsum("nik,nkj->nij", t_after_A[:, :in_channel//2, :], delta1[:, :, :out_channel//2])
    gs2 = torch.einsum("nik,nkj->nij", t_after_A[:, in_channel//2:, :], delta1[:, :, out_channel//2:])
    opacus_utils.create_or_extend_grad_sample(layer.lora_B, torch.cat((gs1, gs2), -1).transpose(-2,-1).contiguous(), batch_dim)
    gs3 = torch.einsum("nik,kj->nij", delta1[:, :, :out_channel//2], layer.lora_B[:lora_b_channel//2, :])
    gs4 = torch.einsum("nik,kj->nij", delta1[:, :, out_channel//2:], layer.lora_B[lora_b_channel//2:, :])
    after_A_deriv = torch.cat((gs3, gs4), -1)
    lora_A_deriv = torch.einsum("nki,nkj->nij", after_A_deriv, layer.lora_dropout(A))
    opacus_utils.create_or_extend_grad_sample(layer.lora_A, lora_A_deriv.contiguous(), batch_dim)

def evaluate(epoch_idx, evaluate_type):
        model.eval()
        if evaluate_type == 'Test':
            loader = test_loader
            dataset = test_set
        else:
            loader = dev_loader
            dataset = dev_set
        with torch.no_grad():
            model.eval()
            hit1, loss = 0, 0
            for x_hs, x_ts in loader:
                if False and self.args.extend_data:
                    _loss, _hit1 = self.model.test_extend_data(x_hs, x_ts)
                elif evaluate_type == 'Test':
                    _loss, _hit1, top10 = model(x_hs, x_ts, return_candidates=True)
                else:
                    _loss, _hit1 = model(x_hs, x_ts)
                hit1 += _hit1
                loss += _loss.item()
            hit1 /= len(dataset)
            print("{} {} Epoch {} Loss: {} Hit@1:".format(args.relation_id, evaluate_type, epoch_idx,
                                                          loss / len(dataset)), hit1)
        return loss, hit1

class AverageMeter(object):
    """Computes and stores the average and current value
         Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [7]:
class Args:
    init_checkpoint = "/content/drive/MyDrive/Differentially-Private-Fine-tuning-of-Language-Models-main/Language-Generation-GPT-2/pretrained_checkpoints/gpt2-pytorch_model.bin"
    learning_rate = 2e-5
    train_batch_size = 8
    valid_batch_size = 4
    grad_acc_steps = 1
    epochs = 3
    noise_multiplier = 1.0
    max_grad_norm = 1.0
    lstm_dropout = 0.1
    hidden_size = 768
    max_length = 128

    relation_id = "P1001"
    model_name = 'gpt2'
    pseudo_token = '[PROMPT]'

    t5_shard = 0
    mid = 0
    template = (3, 3, 3)
    early_stop = 20

    lr = 1e-5
    seed = 34
    decay_rate = 0.98
    weight_decay = 0.0005
    no_cuda = False
    seq_len = 128


    only_evaluate = False
    use_original_template = False
    use_lm_finetune = False

    vocab_strategy = "original"

    train_data = '/content/drive/MyDrive/Differentially-Private-Fine-tuning-of-Language-Models-main/Language-Generation-GPT-2/data/e2e/train.jsonl'
    valid_data = '/content/drive/MyDrive/Differentially-Private-Fine-tuning-of-Language-Models-main/Language-Generation-GPT-2/data/e2e/valid.jsonl'

    # directories
    data_dir = '/content/drive/MyDrive/Differentially-Private-Fine-tuning-of-Language-Models-main/Language-Generation-GPT-2/data/LAMA'
    out_dir = '/content/drive/MyDrive/Differentially-Private-Fine-tuning-of-Language-Models-main/Language-Generation-GPT-2/out/LAMA'


    lora_dim = 4
    lora_alpha = 32
    lora_dropout = 0.0
    label_smooth = 0.1

    device = torch.device("cuda")
    n_gpu = 0 if no_cuda else torch.cuda.device_count()

    assert isinstance(template, tuple)

args = Args()

In [8]:
#tokenizer init
tokenizer = AutoTokenizer.from_pretrained('gpt2', use_fast=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
train_data = FT_Dataset(
    args.train_data, args.train_batch_size, args.seq_len,
    #    joint_lm=args.obj=='jlm'
)

valid_data = FT_Dataset(
    args.valid_data, args.valid_batch_size, args.seq_len,
)

train_loader = DataLoader(
    train_data, batch_size=args.train_batch_size, num_workers=0,
    shuffle=False, pin_memory=False, drop_last=True,
)

valid_loader = DataLoader(
    valid_data, batch_size=args.valid_batch_size, num_workers=0,
    shuffle=False, pin_memory=False, drop_last=False,
)

In [10]:
config = GPT2Config(
    n_embd=768, n_layer=12, n_head=12,
    lora_attn_dim=args.lora_dim,
    lora_attn_alpha=args.lora_alpha,
    lora_dropout=args.lora_dropout,
)
model = GPT2LMModel(config)
#model.to(args.device)

if args.init_checkpoint is not None:
        print('loading model pretrained weight.')
        model.load_weight(torch.load(args.init_checkpoint))

loading model pretrained weight.


In [11]:
class PromptEncoder(torch.nn.Module):
    def __init__(self, template, hidden_size, tokenizer, device, args):
        super().__init__()
        self.device = device
        self.spell_length = sum(template)
        self.hidden_size = hidden_size
        self.tokenizer = tokenizer
        self.args = args
        # ent embedding
        self.cloze_length = template
        self.cloze_mask = [
            [1] * self.cloze_length[0]  # first cloze
            + [1] * self.cloze_length[1]  # second cloze
            + [1] * self.cloze_length[2]  # third cloze
        ]
        self.cloze_mask = torch.LongTensor(self.cloze_mask).bool().to(self.device)

        self.seq_indices = torch.LongTensor(list(range(len(self.cloze_mask[0])))).to(self.device)
        # embedding
        self.embedding = torch.nn.Embedding(len(self.cloze_mask[0]), self.hidden_size).to(self.device)
        # LSTM
        self.lstm_head = torch.nn.LSTM(input_size=self.hidden_size,
                                       hidden_size=self.hidden_size // 2,
                                       num_layers=2,
                                       dropout=self.args.lstm_dropout,
                                       bidirectional=True,
                                       batch_first=True)
        self.lstm_head = DPLSTM(input_size=self.hidden_size,
                                       hidden_size=self.hidden_size // 2,
                                       num_layers=2,
                                       dropout=self.args.lstm_dropout,
                                       bidirectional=True,
                                       batch_first=True)
        self.mlp_head = nn.Sequential(nn.Linear(self.hidden_size, self.hidden_size),
                                      nn.ReLU(),
                                      nn.Linear(self.hidden_size, self.hidden_size))
        print("init prompt encoder...")

    def forward(self):
        input_embeds = self.embedding(self.seq_indices).unsqueeze(0)
        output_embeds = self.mlp_head(self.lstm_head(input_embeds)[0]).squeeze()
        return output_embeds

In [12]:
class GPT2ModelWithSoftPrompts(torch.nn.Module):
    def __init__(self, model, template, hidden_size, tokenizer, device, args):
        super().__init__()
        self.model = model.to(device)
        # Create an instance of PromptEncoder
        template = (template[0], template[1], 0)
        self.template = template
        self.prompt_encoder = PromptEncoder(self.template, hidden_size, tokenizer, device, args)
        self.prompt_encoder = self.prompt_encoder.to(device)

    def forward(self, input_ids, attention_mask, target):
        # Generate the dynamic prompt embeddings using the PromptEncoder
        prompt_embeddings = self.prompt_encoder().to(args.device)
        #print(prompt_embeddings.shape)
        # The prompt_embeddings are expected to have a shape of (batch_size, num_prompt_tokens, hidden_size)
        # Ensure that the prompt_embeddings are properly shaped before concatenation
        prompt_embeddings = prompt_embeddings.unsqueeze(0).repeat(input_ids.shape[0], 1, 1).to(args.device)
        #print(prompt_embeddings.shape)
        # Concatenate the prompt embeddings with the original token embeddings
        inputs_embeds = torch.cat((prompt_embeddings, self.model.transformer.wte(input_ids)), dim=1).to(args.device)

        #print(inputs_embeds.shape)

        # Adjust attention_mask for the added prompts
        prompt_attention_mask = torch.ones((attention_mask.shape[0], prompt_embeddings.shape[1]), device=attention_mask.device).to(args.device)
        full_attention_mask = torch.cat((prompt_attention_mask, attention_mask), dim=1).to(args.device)

        # Process through the model
        outputs = self.model(inputs_embeds=inputs_embeds, attention_mask=full_attention_mask, labels=target)

        return outputs


sp_model = GPT2ModelWithSoftPrompts(model, args.template, 768, tokenizer, args.device, args)

init prompt encoder...


In [34]:
if args.lora_dim > 0:
      lora.mark_only_lora_as_trainable(model)
opacus_utils.register_grad_sampler(MergedLinear)(compute_transformers_MergedLinear_grad_sample)

<function __main__.compute_transformers_MergedLinear_grad_sample(layer: loralib.layers.MergedLinear, A: torch.Tensor, B: torch.Tensor, batch_dim: int = 0) -> None>

In [43]:
params_to_optimize = []
for name, param in sp_model.prompt_encoder.named_parameters():
    if name == "embedding.weight":
            param.requires_grad = True
    else:
      param.requires_grad = True

    if param.requires_grad:
            params_to_optimize.append({'params': param})
            #print(name)

for param in sp_model.model.parameters():
    param.requires_grad = False


for name, param in sp_model.named_parameters():
  if param.requires_grad == True:
    print(name)

prompt_encoder.embedding.weight
prompt_encoder.lstm_head.weight_ih_l0
prompt_encoder.lstm_head.bias_ih_l0
prompt_encoder.lstm_head.weight_hh_l0
prompt_encoder.lstm_head.bias_hh_l0
prompt_encoder.lstm_head.weight_ih_l0_reverse
prompt_encoder.lstm_head.bias_ih_l0_reverse
prompt_encoder.lstm_head.weight_hh_l0_reverse
prompt_encoder.lstm_head.bias_hh_l0_reverse
prompt_encoder.lstm_head.weight_ih_l1
prompt_encoder.lstm_head.bias_ih_l1
prompt_encoder.lstm_head.weight_hh_l1
prompt_encoder.lstm_head.bias_hh_l1
prompt_encoder.lstm_head.weight_ih_l1_reverse
prompt_encoder.lstm_head.bias_ih_l1_reverse
prompt_encoder.lstm_head.weight_hh_l1_reverse
prompt_encoder.lstm_head.bias_hh_l1_reverse
prompt_encoder.mlp_head.0.weight
prompt_encoder.mlp_head.0.bias
prompt_encoder.mlp_head.2.weight
prompt_encoder.mlp_head.2.bias


In [39]:
optimizer = torch.optim.Adam(params_to_optimize, lr=args.lr, weight_decay=args.weight_decay)

In [ ]:
#attaching the privacy engine
ALPHAS = [1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64))
SAMPLE_RATE = (args.train_batch_size * args.grad_acc_steps)/42061.0
privacy_engine = PrivacyEngine(
    module=model,
    sample_rate=SAMPLE_RATE,
    alphas=ALPHAS,
    noise_multiplier=args.noise_multiplier,
    max_grad_norm=args.max_grad_norm,
)
privacy_engine.attach(optimizer)

/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:759: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:236: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(


IncompatibleModuleException: Model contains incompatible modules.
grad_sampler method is not yet supported for this module.: ['_module.transformer.h.0.attn.c_attn (Conv1D)', '_module.transformer.h.0.attn.c_proj (Conv1D)', '_module.transformer.h.0.mlp.c_fc (Conv1D)', '_module.transformer.h.0.mlp.c_proj (Conv1D)', '_module.transformer.h.1.attn.c_attn (Conv1D)', '_module.transformer.h.1.attn.c_proj (Conv1D)', '_module.transformer.h.1.mlp.c_fc (Conv1D)', '_module.transformer.h.1.mlp.c_proj (Conv1D)', '_module.transformer.h.2.attn.c_attn (Conv1D)', '_module.transformer.h.2.attn.c_proj (Conv1D)', '_module.transformer.h.2.mlp.c_fc (Conv1D)', '_module.transformer.h.2.mlp.c_proj (Conv1D)', '_module.transformer.h.3.attn.c_attn (Conv1D)', '_module.transformer.h.3.attn.c_proj (Conv1D)', '_module.transformer.h.3.mlp.c_fc (Conv1D)', '_module.transformer.h.3.mlp.c_proj (Conv1D)', '_module.transformer.h.4.attn.c_attn (Conv1D)', '_module.transformer.h.4.attn.c_proj (Conv1D)', '_module.transformer.h.4.m

In [ ]:
#check if privacy engine is attached - for debug
if hasattr(optimizer, "privacy_engine"):
    print("PrivacyEngine is attached.")
    is_attached = isinstance(optimizer.privacy_engine, PrivacyEngine)
    print(f"PrivacyEngine is correctly attached: {is_attached}")
else:
    print("PrivacyEngine is NOT attached.")

PrivacyEngine is NOT attached.


In [27]:
from torch.nn import CrossEntropyLoss

In [44]:
for name, param in sp_model.named_parameters():
    if not param.requires_grad:
        print(f"Parameter name: {name} is frozen (not trainable).")
    else: print(name)

Parameter name: model.transformer.wte.weight is frozen (not trainable).
Parameter name: model.transformer.wpe.weight is frozen (not trainable).
Parameter name: model.transformer.h.0.ln_1.weight is frozen (not trainable).
Parameter name: model.transformer.h.0.ln_1.bias is frozen (not trainable).
Parameter name: model.transformer.h.0.attn.c_attn.weight is frozen (not trainable).
Parameter name: model.transformer.h.0.attn.c_attn.bias is frozen (not trainable).
Parameter name: model.transformer.h.0.attn.c_attn.lora_A is frozen (not trainable).
Parameter name: model.transformer.h.0.attn.c_attn.lora_B is frozen (not trainable).
Parameter name: model.transformer.h.0.attn.c_proj.weight is frozen (not trainable).
Parameter name: model.transformer.h.0.attn.c_proj.bias is frozen (not trainable).
Parameter name: model.transformer.h.0.ln_2.weight is frozen (not trainable).
Parameter name: model.transformer.h.0.ln_2.bias is frozen (not trainable).
Parameter name: model.transformer.h.0.mlp.c_fc.weigh

In [45]:
for epoch in range(args.epochs):
    total_loss = 0
    sp_model.train()
    avg_lm_loss = AverageMeter()
    with tqdm(total = len(train_loader), desc=f"Epoch {epoch + 1}/{args.epochs}", position=0, leave=True, ncols=80) as progress_bar:
        for idx, data in enumerate(train_loader):

            torch.cuda.empty_cache()
            data = {key: value for key, value in data.items()}
            #print(torch.cuda.memory_summary())
            _input = data['input'].to(args.device)
            _target = data['target'].to(args.device)
            _msk = data['mask'].to(args.device)
            output = sp_model(
                input_ids =_input, attention_mask=_msk, target = None
            )
            lm_logits= output[0]
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = _target[..., 1:].contiguous()
            shift_logits = shift_logits[..., 6:, :].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            loss.backward()

            torch.cuda.empty_cache()
            optimizer.step()
            optimizer.zero_grad()
            #print("Prompt embeddings after update:", model.prompt_embeddings.weight.data)
            torch.cuda.empty_cache()
            total_loss += loss.item()
            progress_bar.set_postfix_str(f"Loss: {loss.item()}")
            progress_bar.update()

    avg_loss = total_loss / len(train_loader)
    print(f"\nEpoch {epoch + 1}/{args.epochs} - Average Loss: {avg_loss}")

Epoch 1/3: 100%|██| 5258/5258 [28:18<00:00,  3.09it/s, Loss: 1.8207974433898926]



Epoch 1/3 - Average Loss: 2.3861988005913815


Epoch 2/3: 100%|██| 5258/5258 [28:10<00:00,  3.11it/s, Loss: 1.6014472246170044]



Epoch 2/3 - Average Loss: 2.0052692187109025


Epoch 3/3: 100%|███| 5258/5258 [28:17<00:00,  3.10it/s, Loss: 1.592934489250183]


Epoch 3/3 - Average Loss: 1.8834533438613634


In [48]:
torch.save({
    'sp_model_state_dict': sp_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, '/content/drive/MyDrive/Differentially-Private-Fine-tuning-of-Language-Models-main/model_checkpoint.pth')

In [49]:
!pip install nltk

In [50]:
from nltk.translate.bleu_score import corpus_bleu

In [53]:
sp_model.eval()
predictions = []
references = []
with torch.no_grad():
    for idx, data in enumerate(valid_loader):
        input_ids = data['input'].to(args.device)
        target_ids = data['target'].to(args.device)
        mask = data['mask'].to(args.device)

        # Generate output using the model
        outputs = sp_model(input_ids=input_ids, attention_mask=mask, target=None)
        lm_logits = outputs[0]

        # Convert logits to token ids
        predicted_token_ids = torch.argmax(lm_logits, dim=-1)

        # Detokenize or decode text here (depending on your tokenizer)
        # This is an example; you'll need to replace it with your actual detokenization method
        predicted_texts = [tokenizer.decode(ids) for ids in predicted_token_ids]
        target_texts = [tokenizer.decode(ids) for ids in target_ids]

        predictions.extend(predicted_texts)
        references.extend([[text] for text in target_texts])  # Note that references should be a list of lists

bleu_score = corpus_bleu(references, predictions)
print(bleu_score)


0.5632400320862083
